## MxCleaner - Untargeted Metabolomics Data Cleaner

Data cleaning criteria based on [Southam et al. 2021](https://pubmed.ncbi.nlm.nih.gov/33236910/)

Input: csv file (here: Agilent Profinder peak area export)

Output: csv file after data cleaning

Either use the following header format for the input file, 
i.e. blank samples start w/ "Blank", samples w/ "S" and QC samples w/ "QC" 

(the header order does not matter)

| Mass | RT   | Blank01 | .. | BlankXX | S01 | .. | SXX | QC01 | .. | QCXX |
|------|------|---------|----|---------|-----|----|-----|------|----|------|
| 101.12 | 0.72 | 2349  |    | 4500    | 398746 |    | 445678 | 500123 |    | 516980 |



Or define column header prefixes here:

In [35]:
# Define column header prefixes
Mass   = "Mass"
RT     = "RT"
Sample = "S","KO","WT" # multiple sample_prefixes = "S", "KO", "WT"
Blank  = "Blank"
QC     = "QC"

### Data Cleaning Steps:

- Concatenate mass w/ Rt

- Remove QC sample if QC total peak area deviates +/-25% of the median QC total peak area

- Remove features if detected in < 70% of QC samples

- Remove features if absent across all sample groups

- Remove duplicate features if present

- Remove features w/ a QC/Blank ratio < 5

- Remove features if CV > 30% in QC samples




In [36]:
import pandas as pd
import numpy as np

In [37]:
# Read csv
file_path = 'example_data/input.csv'
df = pd.read_csv(file_path)

In [38]:
# Concatenate "Mass" and "RT" to "mass_Rt"
df['mass_Rt'] = df[Mass].astype(str) + '_' + df[RT].astype(str)

# Drop "Mass" and "RT"
df.drop(columns=[Mass, RT], inplace=True)
df.shape

(1710, 40)

In [39]:
df.head(5)

,Blank01,Blank02,Blank03,S01,S02,S03,S04,S05,S06,S07,...,S28,S29,S30,QC01,QC02,QC03,QC04,QC05,QC06,mass_Rt
0,NaN,NaN,NaN,NaN,NaN,4.607059e+05,NaN,NaN,NaN,NaN,...,3087209.02,NaN,NaN,5.828618e+06,5.599651e+06,5.927298e+06,6.573955e+06,37694745.93,37204080.59,237.0917_2.75
1,2.924332e+07,8.695783e+07,4.014109e+07,1.124319e+07,9.518030e+06,7.063135e+06,4.557681e+06,NaN,58901931.67,7.267850e+06,...,11383541.21,9.509425e+06,10553410.67,4.542897e+07,4.503961e+07,3.684439e+07,4.346505e+07,43281595.50,42894786.13,101.1209_0.72
2,NaN,NaN,NaN,2.604254e+05,NaN,NaN,NaN,NaN,73861263.33,6.771373e+07,...,60498933.68,6.111982e+07,54154676.61,5.470662e+07,5.426371e+07,5.066745e+07,4.981715e+07,48805049.55,48653615.46,220.1038_2.81
3,3.383789e+04,3.555792e+04,3.156150e+04,4.456561e+07,6.410712e+07,4.698877e+07,5.276094e+07,5.195855e+07,69393960.11,7.540133e+07,...,68003405.84,8.177469e+07,67247843.23,6.380151e+07,6.401953e+07,6.485241e+07,6.601836e+07,66081061.23,66716160.80,612.1532_1.1
4,8.623461e+04,1.035175e+05,1.105659e+05,1.622884e+05,1.117921e+05,1.911437e+05,1.045923e+05,9.465556e+04,67747054.00,6.329350e+07,...,64612869.16,6.598502e+07,NaN,5.715152e+07,5.665737e+07,5.399662e+07,5.347591e+07,52695902.88,52715234.40,239.0899_2.74


### 1. Remove QC sample if QC total peak area deviates +/-25% of the median QC total peak area

In [40]:
## Split df in two dfs, a df with and a df w/o 'QC' samples

# Select columns matching substring 'QC'
df_qc = df.filter(regex='QC')

# Filter columns not starting with 'QC'
df_no_qc = df.loc[:, ~df.columns.str.startswith(QC)]

In [41]:
# Calculate column sums and percentages
sum = df_qc.sum()
pct = sum / sum.median() *100

In [42]:
# Filter columns based on percentages
dev = 25 #define deviation in %
df_qc = df_qc.loc[:, (pct >= (100-dev)) & (pct <= (100+dev))]

In [43]:
df1 = pd.concat([df_no_qc, df_qc], axis=1)
df1.shape

(1710, 40)

### 2. Remove features if detected in <70% of QC samples

In [44]:
df2 = df1.copy()

# Count the number of columns starting with 'QC'
nqc = np.sum(df2.columns.str.startswith(QC))

# Subset the DataFrame to remove features detected in <70% of QC samples
det = 70 #define detection limit in %
df2 = df2[df2.loc[:, df2.columns.str.startswith(QC)].count(axis=1) > nqc * (det/100)]
df2.shape

(1217, 40)

### 3. Remove features if absent across all sample groups

In [45]:
df3 = df2.copy()

# Subset the DataFrame to remove features absent across all sample groups
df3 = df3[df3[df3.columns[df3.columns.str.startswith(tuple(Sample))]].count(axis=1) > 0]
df3.shape

(1217, 40)

### 4. Remove duplicate features if present

In [46]:
df4 = df3.copy()

# Keep only the first occurrence of each value in the specified column
df4['mass_Rt'] = df4['mass_Rt'].loc[~df4['mass_Rt'].duplicated(keep='first')]
df4.shape

(1217, 40)

### 5. Remove features w/ a QC/Blank ratio <5

In [47]:
df5 = df4.copy()

# Calculate mean of QC samples and add in a new column
df5['QCMean'] = df5.filter(regex='QC').mean(axis=1, skipna=True)

# Calculate mean of blank samples and add in a new column
df5['BlankMean'] = df5.filter(regex='Blank').mean(axis=1, skipna=True)

# Calculate ratio QC/blank
df5['Ratio'] = df5['QCMean'] / df5['BlankMean']

# Remove features if ratio < 5, keep rows with NA
ratio = 5 #define ratio
df5 = df5[(df5['Ratio'] >= ratio) | df5['Ratio'].isna()]
df5.shape

(892, 43)

### 6. Remove features if CV >30% in QC samples

In [48]:
df6 = df5.copy()

# Get columns matching substring 'QC'
cols = df6.filter(regex='QC').columns

# Calculate SD of QC samples and add in a new column
df6['QC_SD'] = df6[cols].apply(lambda x: np.std(x, ddof=1), axis=1, raw=True)

# Calculate CV of QC samples and add in a new column
df6['QC_CV'] = (df6['QC_SD'] / df6['QCMean']) * 100

# Remove features if CV > 30% in QC samples
cv = 30 #define CV in %
df6 = df6[df6['QC_CV'] <= cv]
df6.shape

(738, 45)

### 7. Export cleaned dataset

In [49]:
df7 = df6.copy()

# Delete unnecessary columns (matching substring "Blank", "Ratio", "BlankMean", "QCMean", "QC_SD", "QC_CV")
columns_to_drop = df7.filter(regex='Blank|Ratio|BlankMean|QCMean|QC_SD|QC_CV').columns
df7 = df7.drop(columns=columns_to_drop)
df7.shape

(738, 37)

In [50]:
# Move 'mass_Rt' column to the first position
df7 = pd.concat([df7['mass_Rt'], df7.drop('mass_Rt', axis=1)], axis=1)

In [51]:
# Create a new file name with '_cleaned' appended
cleaned_file_path = file_path.replace('.csv', '_cleaned.csv')

# Write the cleaned DataFrame to the new CSV file
df7.to_csv(cleaned_file_path, index=False)